Active earth pressure calculations
==================================

The present notebook allows the calculation of the active earth pressure using the following solutions:
* The kinematic solution proposed in D. Perozzi, A. M. Puzrin, "Limit-state solutions for the active earth pressure behind
    walls rotating about the base", submitted to Géotechnique in 2023.
* The extended Lancellotta solution, also presented in the same paper
* The classical Coulomb's theory

The sign convention follows that of D. Perozzi, A. M. Puzrin, "Limit-state solutions for the active earth pressure behind
    walls rotating about the base", submitted to Géotechnique in 2023.

If a rotation about the wall base is assumed, this utility returns the load coefficient $K_{\text{a},\,n}$, which can be used according to Perozzi and Puzrin (2023) to calculate the moment acting at the wall base as:

$$ m_\text{a} = \frac{\gamma h^3}{6 \cos \alpha} K_{\text{a},\,n} $$

If a translation is assumed, the load coefficient $ K_{\text{a},\,h}$ is returned. It can be used to calculate the total horizontal earth pressure acting on the wall:

$$ E_{\text{a},\,h} = \frac{\gamma h^2}{2} K_{\text{a},\,h} $$


Copyright (c) 2023. ETH Zurich, David Perozzi; D-BAUG; Institute for Geotechnical Engineering; Chair of Geomechanics and Geosystems Engineering

This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
import math

import ipywidgets as widgets
import numpy as np
from IPython.display import display

from earth_pressure_la import kinematic_solution as kis
from earth_pressure_la import static_solution as sts
from misc.definitions import ALPHA, BETA, DELTA, PHI
from misc.definitions import ONE_WEDGE, TWO_WEDGES, EXT_ONE_WEDGE

# ToggleButtons for choosing between rotation about the base and translation
toggle_buttons = widgets.ToggleButtons(
    options=["Rotation about the base", "Horizontal translation"],
    description='Wall failure mode:',
    disabled=False,
    button_style='',
    style={"description_width": "initial"}
)

# Create HTML widgets for the input parameters' name
alpha_label = widgets.HTML(value="<i>&alpha;</i> (&deg;):")
beta_label = widgets.HTML(value="<i>&beta;</i> (&deg;):")
phi_label = widgets.HTML(value="<i>&phi;</i> (&deg;):")
delta_label = widgets.HTML(value="<i>&delta;</i> (&deg;):")

# Create input fields for the input parameters
input_alpha = widgets.Text()
input_beta = widgets.Text()
input_phi = widgets.Text()
input_delta = widgets.Text()

# Create horizontal boxes to hold the HTML label and text input side-by-side
alpha_hbox = widgets.HBox([alpha_label, input_alpha])
beta_hbox = widgets.HBox([beta_label, input_beta])
phi_hbox = widgets.HBox([phi_label, input_phi])
delta_hbox = widgets.HBox([delta_label, input_delta])

# Create a button to trigger the function
button = widgets.Button(description="Calculate")

# Create labels to put the results
output_text = widgets.Label()
output_kin = widgets.Label()
output_lanc = widgets.Label()
output_coulomb = widgets.Label()

output_vbox = widgets.VBox([output_text, output_kin, output_lanc, output_coulomb])

# widget to display the program output (exceptions, warnings, and general output)
output = widgets.Output()
label_output = widgets.Label(value="Output console:")
output_console_vbox = widgets.VBox([label_output, output])


def on_button_click(b):
    with output:
        # Clear output
        output.clear_output()
        output_text.value = ""
        output_kin.value = ""
        output_lanc.value = ""
        output_coulomb.value = ""
        
        # Input validation
        try:
            phi = float(input_phi.value)
            alpha = float(input_alpha.value)
            beta = float(input_beta.value)
            delta = float(input_delta.value)
        except ValueError:
            output_text.value = "Invalid inputs. Please enter numbers only."

        if not (0 <= phi <= 55):
            output_text.value = "Invalid phi: Must be between 0 and 55."
            return
        if not (-45 <= alpha <= 45):
            output_text.value = "Invalid alpha: Must be between -45 and 45."
            return
        if not (-phi <= beta <= phi):
            output_text.value = f"Invalid beta: Must be between -{phi} and {phi}."
            return
        if not (0 <= delta <= phi):
            output_text.value = f"Invalid delta: Must be between 0 and {phi}."
            return
        params = {PHI: math.radians(phi), ALPHA: -math.radians(alpha), BETA: np.array([math.radians(beta)]),
                  DELTA: math.radians(delta)}
        
        if toggle_buttons.value == "Rotation about the base":
            mode = "RF"
        else:
            mode = "T"
        
        # Kinematic solution
        kin_sol = kis.KinematicSolution([ONE_WEDGE, TWO_WEDGES, EXT_ONE_WEDGE], params, mode)
        kin_sol.set_var_parameter(BETA)
        kin_sol.optimize()
    
        # Static approx. solution
        lancellotta_ext = sts.LancellottaExtended()
        lancellotta_ext.set_parameters(phi=math.radians(phi), alpha=-math.radians(alpha), beta=math.radians(beta),
                                       delta=math.radians(delta))
        lancellotta_ext.compute()
        
        # Fill output
        if mode == "RF":
            output_kin.value = "Kinematic solution:\tK_(an) = {:.4f}".format(kin_sol.kan[BETA].max(axis=1).values[0])
            output_coulomb.value = "Coulomb's theory:\tK_(an) = {:.4f}".format(kin_sol.kan_coulomb[BETA][0])
            output_lanc.value = output_text.value + "\nLancellotta extended:\tK_(an) = {:.4f}".format(
                lancellotta_ext.k_n_norm[0])
        else:
            output_kin.value = "Kinematic solution:\tK_(ah) = {:.4f}".format(kin_sol.kah[BETA].max(axis=1).values[0])
            output_coulomb.value = "Coulomb's theory:\tK_(ah) = {:.4f}".format(kin_sol.kah_coulomb[BETA][0])
            output_lanc.value = output_text.value + "\nLancellotta extended:\tK_(ah) = {:.4f}".format(
                lancellotta_ext.k_h_norm[0])
        print("Program completed successfully")

# Bind the button click     event to the function
button.on_click(on_button_click)

# Display widgets
display(toggle_buttons, alpha_hbox, beta_hbox, phi_hbox, delta_hbox, button, output_vbox, output_console_vbox)